In [ ]:
# here you can place default parameters which will be overwritten by the 
# actual parameters in a new cell below
spatial_res = 0.00018
aoi = "POLYGON ((11.519165 41.786673, 11.519165 42.259016, 12.420044 42.259016, 12.420044 41.786673, 11.519165 41.786673))"

In [ ]:
# Configure logging to stderr such that messages appear in notebook
# NOTE: Logging messages from python files which are called from the notebook 
#       will then also appear in the notebook.
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
)

In [ ]:
from pathlib import Path

# This is a fixed path you can use to write the output file to.
# The data written here will end up in a unique location for every job run.
output_dir = Path("/home/jovyan/result-data")

In [ ]:
import shapely.wkt

# aoi is passed as WKT, need bbox
polygon = shapely.wkt.loads(aoi)
bbox = polygon.bounds
bbox

In [ ]:
from datetime import date, timedelta
from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
import xcube.core.maskset as maskset
import xarray as xr

# credentials from your EDC account are automatically available as environment variables
# which are then used by xcube_sh

date_today = date.today()
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04', 'B05', 'B06', 'SCL'],
                         tile_size=[512, 512],
                         bbox=bbox,
                         spatial_res=spatial_res,
                         time_range=[(date_today - timedelta(days=10)).strftime("%Y-%m-%d"), (date_today - timedelta(days=1)).strftime("%Y-%m-%d")])

cube = open_cube(cube_config)

# print basic information here to help with debugging
water_cube = cube.where(maskset.MaskSet(cube.SCL).water)
water_cube

In [ ]:
b_from = water_cube.B04
b_peek = water_cube.B05
b_to = water_cube.B06

wlen_from = b_from.attrs['wavelength']
wlen_peek = b_peek.attrs['wavelength']
wlen_to = b_to.attrs['wavelength']

f = (wlen_peek - wlen_from) / (wlen_to - wlen_from)
mci = (b_peek - b_from) - f * (b_to - b_from)

mci.attrs['long_name'] = 'Maximum Chlorophyll Index'
mci.attrs['units'] = 'unitless'
mci

In [ ]:
result = xr.Dataset({'mci': mci}).mci.isel(time=-1)
logging.info("size: %s", result.size)

In [ ]:
%matplotlib inline
import IPython.display
result.plot.imshow(vmin=0, vmax=0.005, cmap='viridis', figsize=(16, 10))

In [ ]:
filename = date_today.strftime("%Y-%m-%d") + ".tif"
result.rio.to_raster(output_dir / filename, dtype="float32")
logging.info("file %s written", filename)